# Import lib

In [ ]:
from pprint import pp
import pandas as pd
import geopandas as gpd
import folium
import os, shutil
from glob import glob
import re
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.mask import mask
import tempfile
import random

import matplotlib.pyplot as plt

from osgeo import gdal
import numpy as np

import requests
from bs4 import BeautifulSoup
import re
import os

import pytz

from tqdm import tqdm_notebook

import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from tqdm.notebook import tqdm

%matplotlib inline

from zkyhaxpy import io_tools
from zkyhaxpy.gis_tools import convert_hdf_to_geotiff, extract_cloudless_band

from datetime import datetime, timedelta

# Define functions

### get_gregorian_date

In [ ]:
def get_gregorian_date(year, julian_date):
    date = datetime(year,1,1) + timedelta(julian_date-1)
    return datetime.strftime(date, '%Y-%m-%d')


### orbit_timestamp_to_datetime

In [ ]:
def orbit_timestamp_to_datetime(orbit_timestamp_in):
    if len(orbit_timestamp_in) == 0:
        return
    year = int(orbit_timestamp_in[:4])
    julian_date = int(orbit_timestamp_in[4:7])
    dt = get_gregorian_date(year, julian_date)
    tm = f'{orbit_timestamp_in[7:9]}:{orbit_timestamp_in[9:11]}'
    return f'{dt} {tm}'

In [ ]:
DICT_SUBDATASET = {
    'aod_055':'grid1km:Optical_Depth_055',
    'aod_qa':'grid1km:AOD_QA',
    'aod_055_test':'grid1km:Optical_Depth_055',    
    }

list_tile_id = ['h27v07', 'h27v06']

band_shortname = 'aod_055'
dir_out_root = r'C:\Temp\PIER-Blog-20240411\data\aod\translated'
list_year = list(range(2000, 2015))
np.random.shuffle(list_year)
pbar_year = tqdm(list_year)
for year in pbar_year:
    pbar_year.set_description(f'{year}')
    list_file_path_raw = io_tools.get_list_files(fr'C:\Temp\PIER-Blog-20240411\data\aod\raw\{year}', '.hdf$',)
    np.random.shuffle(list_file_path_raw)

    for file_path in tqdm(list_file_path_raw):
        file_name = os.path.basename(file_path)
        tile_id = file_name.split('.')[2]
        if tile_id not in list_tile_id:
            continue
        year = file_name.split('.')[1][1:5]
        
        dir_out = os.path.join(dir_out_root, band_shortname, tile_id, year)

        ds = gdal.Open(file_path)
        if ds is None:
            print(f'File {file_path} is corrupted. Skip')
            continue
        dict_metadata = ds.GetMetadata_Dict()
        orbit_amount = int(dict_metadata['Orbit_amount'])
        orbit_timestamps = dict_metadata['Orbit_time_stamp']
        list_orbit_dttm = [orbit_timestamp_to_datetime(timestamp) for timestamp in orbit_timestamps.split(' ') if len(timestamp) > 0]


        #Create temp geotiff band data & cloud mask before translate to cloudless raster
        with tempfile.TemporaryDirectory() as dir_temp:   
            io_tools.create_folders(dir_temp)
            path_target = os.path.join(dir_temp, 'temp_055.tif')            
            target_subds = DICT_SUBDATASET[band_shortname]
            convert_hdf_to_geotiff(path_in = file_path, path_out=path_target, subds=target_subds)
            
            path_qa = os.path.join(dir_temp, 'temp_qa.tif')
            qa_subds = DICT_SUBDATASET['aod_qa']
            convert_hdf_to_geotiff(path_in = file_path, path_out=path_qa, subds=qa_subds)
            
            for i, orbit_dttm in enumerate(list_orbit_dttm):
                band_id = i + 1
                file_name_out = f'{band_shortname}-{orbit_dttm[:10]}_{orbit_dttm[11:13]}{orbit_dttm[14:16]}.tif'    
                path_out = os.path.join(dir_out, file_name_out)            
                if os.path.exists(path_out):
                    continue
                
                io_tools.create_folders(path_out)            
                #Translate into cloudless raster
                
                arr_target_cloudless = extract_cloudless_band(
                    path_in_target=path_target, 
                    path_in_qa=path_qa, 
                    band_id_target=band_id, 
                    band_id_qa=band_id, 
                    tup_cloud_bits_idx=(0, 3), #For first 3 digits of QA bits
                    str_cloud_clear='001', #For first 3 digits of QA bits must = '001' to be cloudless
                    qa_total_bits=16,
                    path_out=path_out
                )

                
